# Paths and download data

This:

1) Define the external path from where raw data will be downloaded, where they are stored on your local machine and where the processed data will be stored while running the others notebooks.

2) Download the raw data of the 2019/20 fieldcampaign on Plaine Morte from VAW server to your local machine

# Paths 


### Original (raw) data (VAW server)

In [22]:
#Raw datas of Plaine Morte are in the parent directory of this code (i.e. one step above) 

const glaziodir =  "https://people.ee.ethz.ch/~werderm/pm-data/data/"


#const glaziodir = if Sys.islinux()
        #"$(homedir())/itet-stor/glazio/projects/88007-VAW_Plaine_Morte_Faverges/data/"
    #else
        ##Christoph's comment: my itet-stor directory is not in home... So I manually write where it actually is: :S/
        #"S:/glazio/projects/88007-VAW_Plaine_Morte_Faverges/data/"; 
    #end

### Local directory on your own machine

In [23]:
#Raw datas on your own machine are stored on the following (created) directory:
#@__DIR__ return your current local path: where the notebook is running

#data
path_data = mkpath(joinpath(@__DIR__, "../data")) *"/"
# CTD
path_CTD =  mkpath(joinpath(@__DIR__, "../data/CTD")) *"/"
#DC22
path_DC22 = mkpath(joinpath(@__DIR__, "../data/DC22")) *"/"
#lake data
path_lake = mkpath(joinpath(@__DIR__, "../data/lake_data")) *"/"
#SR50
mkpath(joinpath(@__DIR__, "../data/SR50"));
# fieldbook
mkpath(joinpath(@__DIR__, "../data/Salt_injections"))
#channel elevation
mkpath(joinpath(@__DIR__, "../data/Channel_elevation"))

"C:\\Users\\christophe\\Desktop\\home\\GitHub\\PlaineMorteLake_hydro_thermodynamics\\notebooks\\../data/Channel_elevation"

### Local directory of processed data (products) on your own machine

In [24]:
#Processed data (or "product")
mkpath(joinpath(@__DIR__, "../products/"))
mkpath(joinpath(@__DIR__, "../products/plots/"))
mkpath(joinpath(@__DIR__, "../products/plots/calibration_conductivity"))
mkpath(joinpath(@__DIR__, "../products/plots/calibration_temperature"))

path_products = joinpath(@__DIR__, "../products/")
path_plots = path_products *"plots/";
path_results_cali_cond = path_plots * "calibration_conductivity/";
path_results_cali_temp = path_plots * "calibration_temperature/";

# Function `download`

In [25]:
"""
    download_file(url, destination_file, force_download=false)

Download a file, if it has not been downloaded already.
Input:
- url -- url for download
- destination_file -- path of the file to store the download
- force_download -- force the download, even if file is present

Output: nothing

"""
function download_file(url, destination_file, force_download=false)
    if !isfile(destination_file) || force_download
        download(url, destination_file)
    end
    return nothing
end



"""
    download_all()
Download all files related to CTD and DC22 data from glaziodir (VAW server) to your local machine
"""
function download_all(; force=false)
    # traces
    download_file(joinpath(glaziodir,"Salt_injections/Salt_injections.csv"),
                  "../data/Salt_injections/Salt_injections.csv", force)
    
    #channel elevation
    elevation = readdir(joinpath(glaziodir, "Channel_elevation"))
    for f in elevation
            download_file(glaziodir * "Channel_elevation/$f",
                    "../data/Channel_elevation/$f", force)
    end
        


    #CTD
    dates = readdir(joinpath(glaziodir, "CTD")) #Add a step to explore "2019-07-16" folder, and then the csv which is inside....
    ctd = []
    for i in dates
        c = readdir(joinpath(glaziodir, "CTD/$i"))  #keyword are supported in Julia 1.4. example: sort
        push!(ctd, c)
    end

    for (i,j) in zip(dates, 1:length(dates))
        if !isdir(joinpath(@__DIR__, "../data/CTD","$i"))
            mkdir(joinpath(@__DIR__, "../data/CTD","$i"))
        end
        for f in ctd[j]
            download_file(joinpath(glaziodir, "../data/CTD","$i","$f"), joinpath(path_CTD,"$i","$f"))
        end
    end

    #DC22 (pressure)
    pressure = readdir(joinpath(glaziodir, "DC22"))
    for f in pressure
        download_file(glaziodir * "DC22/$f",joinpath(path_data, "DC22/$f"), force)

        #Lake
        lake = readdir(joinpath(glaziodir, "lake_data"))
        for f in lake
            download_file(glaziodir * "lake_data/$f",joinpath(path_lake, "$f"), force)
        end

        #SR50
        SR50 = readdir(joinpath(glaziodir,  "SR50"))
        for f in SR50
            download_file(glaziodir *  "SR50/$f",joinpath(path_data,"SR50/", "$f"), force)
        end
    end
end

download_all

# Download and store in your local environment
If not already done

In [26]:
if isdir(glaziodir) # so this does not error when there is no network
    download_all()
else
    @warn "Not downloading as $glaziodir is not available"
end

┌ Warning: Not downloading as https://people.ee.ethz.ch/~werderm/pm-data/data/ is not available
└ @ Main In[26]:4
